In [3]:
import json
import pandas as pd
from tqdm import tqdm

In [17]:
raw_files = "CDs_and_Vinyl.jsonl"
meta_files = "meta_CDs_and_Vinyl.jsonl"
threshold = 4

user_ids2items = {}
item_id2count = {}

for line in tqdm(open(raw_files, "r"), total=4800000):
    data = json.loads(line)
    if data["user_id"] not in user_ids2items:
        user_ids2items[data["user_id"]] = []
    if data["asin"] not in item_id2count:
        item_id2count[data["asin"]] = 0
    if data["rating"] >= threshold:
        user_ids2items[data["user_id"]].append((data["asin"], data["timestamp"]))
        item_id2count[data["asin"]] += 1

print(len(user_ids2items))
print(len(item_id2count))


4827273it [00:23, 201805.78it/s]                             

1754118
701706


In [18]:
for i in range(10):
    tmp_user_dict = {}
    tmp_item_count = {}
    over5count_items = set()
    for item_id, count in item_id2count.items():
        if count >= 5:
            over5count_items.add(item_id)

    for user_id, items in user_ids2items.items():
        tmp_items = []
        for item, timestamp in items:
            if item in over5count_items:
                tmp_items.append((item, timestamp))
        if len(tmp_items) >= 5:
            tmp_user_dict[user_id] = tmp_items
            for item, timestamp in tmp_items:
                if item not in tmp_item_count:
                    tmp_item_count[item] = 0
                tmp_item_count[item] += 1
            # over5count_user.add(user_id)
    item_id2count = tmp_item_count
    user_ids2items = tmp_user_dict
    print("len(tmp_user_dict):", len(tmp_user_dict))
    print("len(tmp_item_count):", len(tmp_item_count))

len(tmp_user_dict): 125307
len(tmp_item_count): 146782
len(tmp_user_dict): 107518
len(tmp_item_count): 83253
len(tmp_user_dict): 105076
len(tmp_item_count): 77741
len(tmp_user_dict): 104642
len(tmp_item_count): 76820
len(tmp_user_dict): 104568
len(tmp_item_count): 76655
len(tmp_user_dict): 104548
len(tmp_item_count): 76627
len(tmp_user_dict): 104544
len(tmp_item_count): 76622
len(tmp_user_dict): 104544
len(tmp_item_count): 76622
len(tmp_user_dict): 104544
len(tmp_item_count): 76622
len(tmp_user_dict): 104544
len(tmp_item_count): 76622


In [32]:
item_havefeatures = set()
for line in tqdm(open(meta_files, "r"), total=701700):
    data = json.loads(line)
    if data["parent_asin"] not in item_id2count:
        continue
    item_havefeatures.add(data["parent_asin"])

701959it [00:04, 154016.92it/s]                            


In [34]:
userraw_id2model_id, itemraw_id2model_id = {}, {}
user_idx, item_idx = 1, 1
not_have_feature_items, not_have_feature_count = set(), 0
interaction_file = open("cds.csv", 'w', encoding='utf-8')
for user_id, items in user_ids2items.items():
    if user_id not in userraw_id2model_id:
        userraw_id2model_id[user_id] = user_idx
        user_idx += 1
    tmp_str = str(userraw_id2model_id[user_id]) + "\t"
    for item, timestamp in items:
        if item not in item_havefeatures:
            not_have_feature_items.add(item)
            not_have_feature_count += 1
            continue
        if item not in itemraw_id2model_id:
            itemraw_id2model_id[item] = item_idx
            item_idx += 1
        tmp_str += str(itemraw_id2model_id[item]) + " "
    tmp_str = tmp_str[:-1] + "\n"
    interaction_file.write(tmp_str)
interaction_file.close()

print(not_have_feature_count)
print(not_have_feature_items)
with open("user_id2model_id.json", "w", encoding="utf-8") as f:
    json.dump(userraw_id2model_id, f)
with open("item_id2model_id.json", "w", encoding="utf-8") as f:
    json.dump(itemraw_id2model_id, f)

69
{'B000F3T7X8', 'B00004WFJ7', 'B000005DSQ', 'B00000I3VF', 'B000005DSR', 'B00005TQ52'}


In [39]:
# item_id2count
feature_keys = ["title", "main_category", "categories","description"]
CDsID, rawID, title, main_category, categories, description = [], [], [], [], [], []
for line in tqdm(open(meta_files, "r"), total=701700):
    data = json.loads(line)
    if data["parent_asin"] not in item_id2count:
        continue
    rawID.append(data["parent_asin"])
    CDsID.append(itemraw_id2model_id[data["parent_asin"]])
    title.append(data["title"])
    main_category.append(data["main_category"])
    categories.append(",".join(data["categories"]))
    description.append(data["description"][0] if len(data["description"]) > 0 else "")

701959it [00:04, 161316.36it/s]                            


In [40]:
item_featuers = pd.DataFrame({"CDsID": CDsID, "rawID": rawID, "title": title, "main_category": main_category, "categories": categories, "description": description})
item_featuers.to_csv("CDs_features.csv", index=False, sep="\t")